<a href="https://colab.research.google.com/github/julioreis-dev/data_science/blob/main/ApacheSparkExercicio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%bash
mkdir enem_files
# instalar as dependências
apt-get update && apt-get install openjdk-8-jdk-headless -qq > /dev/null

# instalar pyspark
pip install -q pyspark

# Preparando uma pasta e baixando e descompactando o arquivo
mkdir enem_files
curl --insecure https://download.inep.gov.br/microdados/microdados_enem_2020.zip -o enem_files/microdados.zip
unzip enem_files/microdados.zip -d enem_files/
rm enem_files/microdados.zip

In [7]:
import os
from pyspark.sql import SparkSession
from pyspark.sql import functions as functions

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
spark = SparkSession.builder.getOrCreate()

In [ ]:
enem = (
    spark
    .read
    .csv("enem_files/DADOS/", header=True, sep=";", inferSchema=True, encoding='ISO-8859-1')
)
enem.printSchema()

In [11]:
enem.show()

+------------+------+---------------+-------+---------------+-----------+----------------+---------------+---------------+---------+---------+------------+----------------+-------------------+---------+---------+----------------------+------------------+---------------+------------------+--------------------+-----------+-----------+--------------+--------------+--------------+--------------+-----------+-----------+-----------+-----------+----------+----------+----------+----------+--------------------+--------------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+-----------------+-------------+-------------+-------------+-------------+-------------+---------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|NU_INSCRICAO|NU_ANO|TP_FAIXA_ETARIA|TP_SEXO|TP_ESTADO_CIVIL|TP_COR_RACA|TP_NACIONALIDADE|TP_ST_CONCLUSAO|

In [12]:
enem.createOrReplaceTempView("enem_microdados")

In [13]:
# Qual é a média da nota em Ciências Humanas dos alunos que estudaram numa escola na cidade de Natal?
spark.sql("""
  SELECT
    NO_MUNICIPIO_ESC, AVG(NU_NOTA_CH) as Media_humanas
  FROM enem_microdados
  WHERE NO_MUNICIPIO_ESC = 'Natal' AND NU_NOTA_CH IS NOT NULL
  GROUP BY NO_MUNICIPIO_ESC
""").show()

+----------------+-----------------+
|NO_MUNICIPIO_ESC|    Media_humanas|
+----------------+-----------------+
|           Natal|548.1773212818406|
+----------------+-----------------+



In [15]:
# Qual é a nota média em Ciências Humanas dos alunos que estudaram numa escola no estado de Santa Catarina e possuem PELO MENOS 1 banheiro em casa?
spark.sql("""
  SELECT
    AVG(NU_NOTA_CH) as Media_humanas
  FROM enem_microdados
  WHERE SG_UF_ESC = 'SC' AND NU_NOTA_CH IS NOT NULL AND Q008 not in ('A')
""").show()

+-----------------+
|    Media_humanas|
+-----------------+
|545.0439967817797|
+-----------------+



In [16]:
# Sobre o ENEM 2020, qual é a média da nota em matemática apenas para as alunas do sexo Feminino?
spark.sql("""
  SELECT
    AVG(NU_NOTA_MT) AS Media_matematica
  FROM
    enem_microdados
  WHERE TP_SEXO = 'F' AND NU_NOTA_MT IS NOT NULL
""").show()

+-----------------+
| Media_matematica|
+-----------------+
|500.6864256785913|
+-----------------+



In [17]:
# Qual é a nota média em matemática dos alunos cuja mãe possui PELO MENOS o ensino superior completo, do sexo feminino que estudaram numa escola em Belo Horizonte?
spark.sql("""
  SELECT
    AVG(NU_NOTA_MT) AS Media_matematica
  FROM
    enem_microdados
  WHERE TP_SEXO = 'F' AND NU_NOTA_MT IS NOT NULL AND Q002 IN ('F','G') AND NO_MUNICIPIO_ESC = 'Belo Horizonte'
""").show()

+-----------------+
| Media_matematica|
+-----------------+
|667.1279782501238|
+-----------------+



In [19]:
# Quantas pessoas que estudaram numa escola no estado do Rio de Janeiro do sexo masculino obtiveram nota em matemática maior do que 600?
spark.sql("""
  SELECT
    COUNT(*)
  FROM
    enem_microdados
  WHERE
    TP_SEXO = 'M' AND SG_UF_ESC = 'RJ' AND NU_NOTA_MT > 600

""").show()

+--------+
|count(1)|
+--------+
|    8578|
+--------+

